In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [4]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import timm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/400"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 120  # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

import timm

# Create model
model = timm.create_model("hf_hub:timm/dm_nfnet_f0.dm_in1k", pretrained=True)
model.to(device)  # Move model to GPU

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.8 MB/s eta 0:00:00
Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

Epoch 1/10, Train Loss: 2.9231, Train Accuracy: 35.87%, Val Loss: 1.5778, Val Accuracy: 45.42%


...

Epoch 6/10, Train Loss: 0.4547, Train Accuracy: 83.36%, Val Loss: 0.6375, Val Accuracy: 75.64%


Epoch 7/10, Train Loss: 0.3716, Train Accuracy: 86.11%, Val Loss: 0.8133, Val Accuracy: 74.18%


Epoch 8/10, Train Loss: 0.2690, Train Accuracy: 90.27%, Val Loss: 0.8745, Val Accuracy: 74.73%


Epoch 9/10, Train Loss: 0.2224, Train Accuracy: 91.52%, Val Loss: 0.7349, Val Accuracy: 79.30%
Epoch 00009: reducing learning rate of group 0 to 1.0000e-05.


Epoch 10/10, Train Loss: 0.1417, Train Accuracy: 94.98%, Val Loss: 0.5541, Val Accuracy: 83.52%
Test Accuracy: 83.52%


In [5]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.725     0.935     0.817        31
          PT      0.923     0.800     0.857        30
          PC      0.929     0.531     0.675        49
          MC      0.857     0.692     0.766        52
          LC      0.800     0.545     0.649        44
           F      0.964     0.761     0.850        71
          DC      0.805     0.978     0.883       232
           A      0.857     0.973     0.911        37

    accuracy                          0.835       546
   macro avg      0.858     0.777     0.801       546
weighted avg      0.847     0.835     0.827       546



In [6]:
# Train the model
num_epochs = 20
for epoch in range(10, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 11/20, Train Loss: 0.1105, Train Accuracy: 95.92%, Val Loss: 0.5594, Val Accuracy: 81.68%


Epoch 12/20, Train Loss: 0.0915, Train Accuracy: 96.94%, Val Loss: 0.5645, Val Accuracy: 82.78%


Epoch 13/20, Train Loss: 0.0767, Train Accuracy: 97.17%, Val Loss: 0.5763, Val Accuracy: 82.60%


Epoch 14/20, Train Loss: 0.0604, Train Accuracy: 98.19%, Val Loss: 0.6325, Val Accuracy: 83.33%
Epoch 00014: reducing learning rate of group 0 to 1.0000e-06.


Epoch 15/20, Train Loss: 0.0554, Train Accuracy: 98.27%, Val Loss: 0.6295, Val Accuracy: 83.15%


Epoch 16/20, Train Loss: 0.0538, Train Accuracy: 98.19%, Val Loss: 0.6138, Val Accuracy: 83.52%


Epoch 17/20, Train Loss: 0.0620, Train Accuracy: 98.04%, Val Loss: 0.5911, Val Accuracy: 83.88%


Epoch 18/20, Train Loss: 0.0551, Train Accuracy: 98.27%, Val Loss: 0.5878, Val Accuracy: 83.70%
Epoch 00018: reducing learning rate of group 0 to 1.0000e-07.


Epoch 19/20, Train Loss: 0.0581, Train Accuracy: 97.96%, Val Loss: 0.5880, Val Accuracy: 83.70%


Epoch 20/20, Train Loss: 0.0584, Train Accuracy: 98.19%, Val Loss: 0.5891, Val Accuracy: 83.70%
Test Accuracy: 83.70%


In [7]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.698     0.968     0.811        31
          PT      0.893     0.833     0.862        30
          PC      0.962     0.510     0.667        49
          MC      0.860     0.712     0.779        52
          LC      0.767     0.523     0.622        44
           F      0.982     0.761     0.857        71
          DC      0.807     0.974     0.883       232
           A      0.902     1.000     0.949        37

    accuracy                          0.837       546
   macro avg      0.859     0.785     0.804       546
weighted avg      0.850     0.837     0.828       546



In [8]:
# Train the model
num_epochs = 30
for epoch in range(20, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 21/30, Train Loss: 0.0553, Train Accuracy: 98.04%, Val Loss: 0.5897, Val Accuracy: 83.70%


Epoch 22/30, Train Loss: 0.0578, Train Accuracy: 98.19%, Val Loss: 0.5903, Val Accuracy: 83.70%
Epoch 00022: reducing learning rate of group 0 to 1.0000e-08.


Epoch 23/30, Train Loss: 0.0551, Train Accuracy: 98.27%, Val Loss: 0.5904, Val Accuracy: 83.70%


Epoch 24/30, Train Loss: 0.0531, Train Accuracy: 98.19%, Val Loss: 0.5904, Val Accuracy: 83.70%


Epoch 25/30, Train Loss: 0.0613, Train Accuracy: 98.12%, Val Loss: 0.5904, Val Accuracy: 83.70%


Epoch 26/30, Train Loss: 0.0543, Train Accuracy: 98.51%, Val Loss: 0.5904, Val Accuracy: 83.70%


Epoch 27/30, Train Loss: 0.0561, Train Accuracy: 98.35%, Val Loss: 0.5904, Val Accuracy: 83.70%


Epoch 28/30, Train Loss: 0.0522, Train Accuracy: 98.35%, Val Loss: 0.5904, Val Accuracy: 83.70%


Epoch 29/30, Train Loss: 0.0508, Train Accuracy: 98.59%, Val Loss: 0.5904, Val Accuracy: 83.70%


Epoch 30/30, Train Loss: 0.0538, Train Accuracy: 98.51%, Val Loss: 0.5905, Val Accuracy: 83.70%
Test Accuracy: 83.70%


In [9]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.698     0.968     0.811        31
          PT      0.893     0.833     0.862        30
          PC      0.962     0.510     0.667        49
          MC      0.860     0.712     0.779        52
          LC      0.767     0.523     0.622        44
           F      0.982     0.761     0.857        71
          DC      0.807     0.974     0.883       232
           A      0.902     1.000     0.949        37

    accuracy                          0.837       546
   macro avg      0.859     0.785     0.804       546
weighted avg      0.850     0.837     0.828       546

